# Exercise 03.b: Stability of the oscillation equation

**Due:** _15 December 2020_

**Names:** Linus, Lukas, Mariana, David

Considering the following oscillation equation
$$\frac{{\rm d}q}{{\rm d}t}+i f q = 0$$
where $f$ is the Coriolis parameter. Its analytic solution is $q=q_{0} {\rm e}^{-ift}$.

1. Discretize the oscillation equation using forward Euler, backward Euler and leap-frog schemes and analyse their stability, respectively.

1. With initial value $q_0 = {\rm e}^{i{\pi}/4}$, and $f=10^{-4}s^{-1}$ and $\Delta t=100s$, attempt to solve the oscillation equation by forward Euler, backward Euler and leap-frog schemes, respectively. Integrate the numerical equations for about ten periods of oscillation and compare the numerical solutions with analytic solution.

1. Take $\Delta t=200, 500 \text{ and } 1000s$, repeat the integration, and compare them with analytic solution.


In [ ]:
import numpy as np

## 2.)

In [ ]:
#Inititializing 
f   = 0.0001                 #s⁻¹
q0  = np.exp(np.pi*0.25j)
RHS = f*1j                  
to_solve = Linear_Solver(RHS,q0)

In [ ]:
to_solve.solve()

In [ ]:
class Linear_Solver:
    def __init__(self, RHS, y0, params = None):
        if params is None:
            params  = {'N': 10, 'dt': 100}
        self.dt  = params['dt']
        self.N  = params['N']
        self.RHS = RHS
        self.y0  = y0
        self.keywords()
    
    def keywords(self):
        self.key_dict = {'fw_euler': self.Euler_foreward,'bw_euler': self.Euler_backward,'leapfrog': self.Leap_frog}
    
    def Euler_foreward(self,q_0):
        q_1 = (1-2*self.dt*self.RHS)*q_0
        return(q_1)

    def Euler_backward(self,q_0):
        q_1 = q_0/(1+self.dt*self.RHS)
        return(q_1)

    def Leap_frog(self,q_01):
        q_2 = q_01[0]-2*self.dt*self.RHS*q_01[1]
        return(q_2)

    def solve(self,method='fw_euler', N=None, dt=None):
        if N is not None:
            self.N  = N
        if dt is not None:
            self.dt = dt
        
        scheme = self.key_dict[method]
        y = [self.y0]
        for i in range(self.N):
            step = scheme(y[i])
            y.append(step)
        return(y)